In [1]:
import scipy
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler ## min max scaling은 데이터에 노이즈(이상치)가 없을 때 좋음. standard scaling은 0을 중심으로 scaling
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt", sep='\t')

Y = data['Y']
X = data.drop(columns=['Y'])
X = pd.get_dummies(X, columns=['SEX']) ## 성별은 categorical 변수인데 1과 2로 되어 있으면 2가 더 큰 변수로 간주된다. 따라서 man, woman으로 컬럼을 분할하고 해당하는 성별은 1 그렇지 않으면 0으로 구성.

## 인덱스를 추출해서 데이터셋을 분할하는 것이 훨씬 효과적

idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

## Ridge에서는 반드시 스케일링을 적용해줘야만 한다.
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

>>>> # of Train data : 309
>>>> # of valid data : 133


In [3]:
def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print(f'R-squared:  {metrics.r2_score(y, clf.predict(X)):.6f},    Adjusted R-squared:  {adj_r2_score(clf, X, y):.6f},    MSE: {sse(clf, X, y):.1f}')

In [4]:
penalties = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9, 1, 10]

for penalty in penalties:
    model = Ridge(alpha=penalty).fit(X_scaled.iloc[train_idx], Y.iloc[train_idx])
    score = model.score(X_scaled.iloc[valid_idx], Y.iloc[valid_idx]) ## R square
    pred_y = model.predict(X_scaled.iloc[valid_idx])
    mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
    print(f"Alpah : {penalty:.5f}, R2 : {score:.7f}, MSE : {mse:.7f}, RMSE : {np.sqrt(mse):.7f}")

Alpah : 0.00001, R2 : 0.5301655, MSE : 3084.6095744, RMSE : 55.5392616
Alpah : 0.00005, R2 : 0.5301672, MSE : 3084.5988322, RMSE : 55.5391648
Alpah : 0.00010, R2 : 0.5301692, MSE : 3084.5854446, RMSE : 55.5390443
Alpah : 0.00100, R2 : 0.5302048, MSE : 3084.3519225, RMSE : 55.5369420
Alpah : 0.01000, R2 : 0.5304637, MSE : 3082.6522133, RMSE : 55.5216373
Alpah : 0.10000, R2 : 0.5304511, MSE : 3082.7343410, RMSE : 55.5223769
Alpah : 0.30000, R2 : 0.5294946, MSE : 3089.0142040, RMSE : 55.5789007
Alpah : 0.50000, R2 : 0.5285641, MSE : 3095.1232917, RMSE : 55.6338323
Alpah : 0.60000, R2 : 0.5280578, MSE : 3098.4477026, RMSE : 55.6637018
Alpah : 0.70000, R2 : 0.5275205, MSE : 3101.9751863, RMSE : 55.6953785
Alpah : 0.90000, R2 : 0.5263592, MSE : 3109.5989675, RMSE : 55.7637783
Alpah : 1.00000, R2 : 0.5257398, MSE : 3113.6657269, RMSE : 55.8002305
Alpah : 10.00000, R2 : 0.4513724, MSE : 3601.9109235, RMSE : 60.0159222


In [5]:
model_best = Ridge(alpha=0.01).fit(X_scaled.iloc[train_idx], Y.iloc[train_idx])
summary(model_best, X_scaled.iloc[valid_idx], Y.iloc[valid_idx], xlabels = X_scaled.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept    3.351510  3.802827e+08+0.000000e+00j  0.0000-0.0000j  1.000000
AGE         -14.279970  2.355360e+01+0.000000e+00j -0.6063+0.0000j  0.545373
BMI         127.331685  3.172437e+01+0.000000e+00j  4.0137-0.0000j  0.000100
BP           67.279481  2.797563e+01+0.000000e+00j  2.4049-0.0000j  0.017563
S1         -203.277222  1.635086e+02+0.000000e+00j -1.2432+0.0000j  0.215990
S2          141.209566  1.149112e+02+0.000000e+00j  1.2289-0.0000j  0.221311
S3           10.619292  7.291556e+01+0.000000e+00j  0.1456-0.0000j  0.884429
S4           13.868910  5.708001e+01+0.000000e+00j  0.2430-0.0000j  0.808403
S5          198.003880  5.019348e+01+0.000000e+00j  3.9448-0.0000j  0.000129
S6           19.251667  3.401033e+01-0.000000e+00j  0.5661+0.0000j  0.572318
SEX_1        10.473795  3.802827e+08+0.000000e+00j  0.0000-0.0000j  1.000000
SEX_2       -10.473795  3.802827e+08-0.000000e+00j -0.0000-0.0

In [6]:
"""
모델명 뒤에 CV가 붙은 것은 Cross Validation을 말함. 즉, fold수 만큼 cross validation을 수행하고
그 중 가장 성능이 좋은 모델 하나를 반환하는 형태임.
"""

ridge_cv=RidgeCV(alphas=penalties, cv=5)
model = ridge_cv.fit(X_scaled.iloc[train_idx], Y.iloc[train_idx])
print(f"Best Alpha: {model.alpha_:.4f}, R2:{model.best_score_:.4f}")

Best Alpha: 0.9000, R2:0.4419


In [7]:
model_best = Ridge(alpha=model.alpha_).fit(X_scaled.iloc[train_idx], Y.iloc[train_idx])
score = model_best.score(X_scaled.iloc[valid_idx], Y.iloc[valid_idx])

pred_y = model_best.predict(X_scaled.iloc[valid_idx])
mse = np.sqrt(mean_squared_error(Y.iloc[valid_idx], pred_y))

print(f"Alpha:{0.01}, R2:{score:.7f}, MSE:{mse:.7f}, RMSE:{np.sqrt(mse):.7f}")
summary(model_best, X_scaled.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scaled.columns)

Alpha:0.01, R2:0.5263592, MSE:55.7637783, RMSE:7.4675149
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   42.643650  3.819412e+08+1.464058e+00j  0.0000-0.0000j  1.000000
AGE         -10.492753  2.368453e+01+1.956490e-01j -0.4430+0.0037j  0.658486
BMI         121.677019  3.185811e+01+1.285190e-01j  3.8193-0.0154j  0.000205
BP           66.649695  2.808212e+01+2.461690e-01j  2.3732-0.0208j  0.019072
S1          -21.091765  1.642219e+02+2.280180e-01j -0.1284+0.0002j  0.898001
S2           -5.450923  1.153438e+02+5.460120e-01j -0.0473+0.0002j  0.962379
S3          -55.208206  7.324488e+01-1.457720e-01j -0.7537-0.0015j  0.452344
S4           17.858217  5.734113e+01+1.642880e-01j  0.3114-0.0009j  0.755960
S5          119.080753  5.042164e+01+8.524500e-02j  2.3617-0.0040j  0.019654
S6           22.603065  3.409545e+01+2.738940e-01j  0.6629-0.0053j  0.508542
SEX_1        10.489062  3.819412e+08+1.233104e+00j  0.0000-0.0000j  1.000000
SEX_2